In [27]:
import pandas as pd
import numpy as np

breast_data = pd.read_csv('datasets/breast-w.csv')

cardio_data = pd.read_csv('datasets/Cardiotocography.csv')

In [28]:
breast_data.head(5)

,d1,d2,d3,d4,d5,d6,d7,d8,d9,class
0,5.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,0.0
1,5.0,4.0,4.0,5.0,7.0,10.0,3.0,2.0,1.0,0.0
2,3.0,1.0,1.0,1.0,2.0,2.0,3.0,1.0,1.0,0.0
3,6.0,8.0,8.0,1.0,3.0,4.0,3.0,7.0,1.0,0.0
4,4.0,1.0,1.0,3.0,2.0,1.0,3.0,1.0,1.0,0.0


In [29]:
cardio_data.head(5)

,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,...,d13,d14,d15,d16,d17,d18,d19,d20,d21,class
0,128,0,219,2,34,2.5,0,4.0,2,0,...,54,199,11,1,75,99,102,148,-1,1
1,128,0,2,8,65,2.5,0,0.0,6,0,...,63,192,6,0,104,94,115,28,0,1
2,128,0,16,2,34,2.4,0,3.6,2,0,...,54,199,11,1,75,98,86,144,-1,1
3,150,0,0,0,76,0.3,31,5.4,0,0,...,146,157,1,0,154,153,155,0,1,1
4,141,0,0,0,61,0.4,29,5.3,0,0,...,135,148,1,0,144,143,145,0,1,1


Insert missing values to both datasets!

In [30]:
from generate_missing_values import generate_mcar_missing_values

# Find the feature's names.
cardio_cols = [i for i in list(cardio_data.columns) if i != 'class']
breast_cols = [i for i in list(breast_data.columns) if i != 'class']

#Only inser missing data at features!
cardio_data.loc[:,cardio_cols] = generate_mcar_missing_values(cardio_data.loc[:,cardio_cols], 0.25).copy()

breast_data.loc[:,breast_cols] = generate_mcar_missing_values(breast_data.loc[:,breast_cols], 0.25).copy()


import autosklearn.classification
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)

In [40]:
from autosklearn.classification import AutoSklearnClassifier
automl = AutoSklearnClassifier()
automl.fit(cardio_data.loc[:,cardio_cols], cardio_data.loc[:,'class'])

Exception ignored in: <function _releaseLock at 0x7f6ec98925e0>
Traceback (most recent call last):
  File "/home/aero/anaconda3/envs/smacenv/lib/python3.8/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
